---
# Initial Imports

In [ ]:
import os

print(os.getcwd())
os.chdir("../src/")
print(os.getcwd())

In [ ]:
import random
import time as t
from importlib import reload

import matplotlib
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

import build_model
import centrality
import centrality_utils
import utils
import weight_functions

In [ ]:
binmat = np.array(
    [
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [0, 1, 1],
        [0, 1, 0],
        [0, 0, 1],
        [1, 1, 1],
        [1, 1, 0],
        [1, 0, 1],
    ],
    dtype=np.int8,
)

idx_worklist = np.array(
    [
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-2, -1, -1],
        [-2, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
    ],
    dtype=np.int8,
)

conds_worklist = np.array(
    [
        [0, 1, 2],
        [0, 1, 2],
        [0, 1, 2],
        [2, 0, 1],
        [1, 2, -1],
        [1, -1, -1],
        [2, -1, -1],
        [1, 0, 2],
        [1, 0, -1],
        [0, 2, -1],
    ],
    dtype=np.int8,
)

colarr = np.array(["A", "B", "C"], dtype="<U24")
n_diseases = colarr.shape[0]

print(binmat)
print(conds_worklist)

## Compute hyperedge and hyperarc weights

In [ ]:
inc_mat, weights, prev_arrs = build_model.compute_weights(
    binmat,
    conds_worklist,
    idx_worklist,
    colarr,
    "progression",  # "exclusive",
    weight_functions.modified_sorensen_dice_coefficient,
    utils.compute_progset,
    dice_type=1,
    plot=True,
    ret_inc_mat=None,
)

In [ ]:
inc_mat, weights, prev_arrs = build_model.compute_weights(
    binmat,
    conds_worklist,
    idx_worklist,
    colarr,
    "progression",
    weight_functions.modified_sorensen_dice_coefficient,
    utils.compute_pwset_progset,
    dice_type=1,
    plot=True,
    ret_inc_mat=None,
)

In [ ]:
inc_mat, weights, prev_arrs = build_model.compute_weights(
    binmat,
    conds_worklist,
    idx_worklist,
    colarr,
    "progression",
    weight_functions.modified_sorensen_dice_coefficient,
    utils.compute_simple_progset,
    dice_type=1,
    plot=True,
    ret_inc_mat=None,
)

# Compute Successor and Predecessor Node PageRank Centrality

## Compute node and edge degree matrices

In [ ]:
inc_mat, weights, prev_arrs = build_model.compute_weights(
    binmat,
    conds_worklist,
    idx_worklist,
    colarr,
    "progression",
    weight_functions.modified_sorensen_dice_coefficient,
    utils.compute_progset,
    dice_type=1,
    plot=False,
    ret_inc_mat=None,
)

hyperedge_weights = np.array(weights[0]["weight"])
hyperedge_titles = np.array(weights[0]["disease set"])

hyperarc_weights = np.array(weights[1]["weight"])
hyperarc_titles = np.array(weights[1]["progression"])

node_weights = np.array(weights[2]["weight"])
node_titles = np.array(weights[2]["node"])

In [ ]:
output = build_model.setup_vars(
    inc_mat, n_diseases, hyperarc_weights, hyperarc_titles, node_weights
)
inc_mat_data, hyperarc_data, node_weights, node_degs, edge_degs = output

inc_mat_tail, inc_mat_head = inc_mat_data
edge_weights, hyperarc_titles = hyperarc_data
node_degree_tail, node_degree_head = node_degs
edge_degree_tail, edge_degree_head = edge_degs

## Successor Detection

In [ ]:
P_node = centrality.comp_ptm_std(
    inc_mat_tail, inc_mat_head, edge_weights, node_degree_tail, None, "successor", eps=0
)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
sns.heatmap(P_node, cmap="inferno", ax=ax, annot=True, annot_kws={"fontsize": 16})
ax.set_yticklabels(colarr, fontsize=15, rotation=0)
ax.set_xticklabels(colarr, fontsize=15, rotation=45)
ax.set_title("Successor Transition Matrix", fontsize=18, pad=20)
ax.xaxis.tick_top()
plt.show()

In [ ]:
inpt = (
    (inc_mat_tail, inc_mat_head),
    (edge_weights, node_weights),
    node_degs,
    edge_degree_tail,
)
node_pagerank = centrality.pagerank_centrality(
    inpt,
    rep="standard",
    typ="successor",
    tolerance=1e-6,
    max_iterations=1000,
    is_irreducible=False,
    weight_resultant=False,
    random_seed=None,
    eps=0.0001,
)

all_node_sc_evc = pd.DataFrame(
    {
        "Disease": colarr,
        "PageRank Centrality": node_pagerank,
    }
)
print(f"PageRank sum: {node_pagerank.sum()}")
succ_order = (
    all_node_sc_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .Disease
)
all_node_sc_evc.sort_values(by="PageRank Centrality", ascending=False).reset_index(
    drop=True
).round({"PageRank Centrality": 3})

## Predecessor Detection

In [ ]:
P_pred_node = centrality.comp_ptm_std(
    inc_mat_tail,
    inc_mat_head,
    edge_weights,
    node_degree_head,
    edge_degree_tail,
    rep="predecessor",
)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
sns.heatmap(P_pred_node, cmap="inferno", ax=ax, annot=True, annot_kws={"fontsize": 16})
ax.set_yticklabels(colarr, fontsize=15, rotation=0)
ax.set_xticklabels(colarr, fontsize=15, rotation=45)
ax.set_title("Predecessor Transition Matrix", fontsize=18, pad=20)
ax.xaxis.tick_top()
plt.show()

In [ ]:
inpt = (
    (inc_mat_tail, inc_mat_head),
    (edge_weights, node_weights),
    node_degs,
    edge_degree_tail,
)
node_pagerank = centrality.pagerank_centrality(
    inpt,
    rep="standard",
    typ="predecessor",
    tolerance=1e-6,
    max_iterations=100,
    is_irreducible=False,
    weight_resultant=False,
    random_seed=None,
    eps=0.0001,
)

all_node_pr_evc = pd.DataFrame(
    {
        "Disease": colarr,
        "PageRank Centrality": node_pagerank,
    }
)
print(f"PageRank sum: {node_pagerank.sum()}")
pred_order = (
    all_node_pr_evc.sort_values(by="PageRank Centrality", ascending=False)
    .reset_index(drop=True)
    .Disease
)
all_node_pr_evc.sort_values(by="PageRank Centrality", ascending=False).reset_index(
    drop=True
).round({"PageRank Centrality": 3})

# Compute Undirected Representation of Directed Hypergraph

## Node Adjacency Matrix

In [ ]:
# Construct incidence matrix for undirected representation of directed hypergraph
incidence_matrix = np.concatenate([inc_mat_tail, inc_mat_head], axis=0)

In [ ]:
node_adj = centrality.comp_adjacency(
    incidence_matrix,
    hyperarc_weights,
    node_weights,
    rep="standard",
    weight_resultant=True,
)

fig, ax = plt.subplots(1, 1, figsize=(10, 7))
sns.heatmap(node_adj, cmap="inferno", ax=ax, annot=True)
ax.set_yticks(ax.get_yticks(), node_titles, fontsize=12, rotation=0)
ax.set_xticklabels(node_titles, fontsize=12, rotation=45, ha="left")
ax.xaxis.tick_top()
plt.show()

In [ ]:
node_centrality = centrality.eigenvector_centrality(
    incidence_matrix,
    hyperarc_weights,
    node_weights,
    rep="standard",
    tolerance=1e-6,
    max_iterations=1000,
    weight_resultant=True,
    random_seed=None,
)
node_evc = pd.DataFrame(
    {
        "Disease": node_titles,
        "Eigenvector Centrality": node_centrality,
    }
)
node_evc.sort_values(by="Eigenvector Centrality", ascending=False).reset_index(
    drop=True
)

## Edge Adjacency Matrix

### Connective Adjacency

In [ ]:
edge_adj = centrality.comp_adjacency(
    incidence_matrix, hyperarc_weights, node_weights, rep="dual", weight_resultant=True
)

fig, ax = plt.subplots(1, 1, figsize=(10, 7))
sns.heatmap(edge_adj, cmap="inferno", ax=ax, annot=True)
ax.set_yticks(ax.get_yticks(), hyperarc_titles, fontsize=12, rotation=0)
ax.set_xticklabels(hyperarc_titles, fontsize=12, rotation=45, ha="left")
ax.xaxis.tick_top()
plt.show()

In [ ]:
hyperarc_centrality = centrality.eigenvector_centrality(
    incidence_matrix,
    hyperarc_weights,
    node_weights,
    rep="dual",
    tolerance=1e-6,
    max_iterations=1000,
    weight_resultant=False,
    random_seed=None,
)

n_conds = n_diseases * [1] + [
    len(d.split(",")) + 1 for d in hyperarc_titles[n_diseases:]
]
hyperarc_evc = pd.DataFrame(
    {
        "Disease": hyperarc_titles,
        "Degree": n_conds,
        "Eigenvector Centrality": hyperarc_centrality,
    }
)
hyperarc_evc.sort_values(by="Eigenvector Centrality", ascending=False).reset_index(
    drop=True
)

### Progressive Adjacency

In [ ]:
# This includes the head node into the tail to convert hyperarc A, B -> C to A, B, C -> C. This is so the hyperarc
# adjacency matrix will now measure strength of connection by not only matching the tail-head combination of hyperarcs
# together, but also matching those hyperarcs which feed into higher degree hyperarcs too. Note the difference in
# adjacency strength between A -> B and A, B -> C in the following setup in comparison to above.
new_inc_mat_tail = inc_mat_tail + inc_mat_head
new_inc_mat_tail[new_inc_mat_tail > 1] = 1
edge_incidence_matrix = np.concatenate([new_inc_mat_tail, inc_mat_head], axis=0)

In [ ]:
edge_adj = centrality.comp_adjacency(
    edge_incidence_matrix,
    hyperarc_weights,
    node_weights,
    rep="dual",
    weight_resultant=True,
)

fig, ax = plt.subplots(1, 1, figsize=(10, 7))
sns.heatmap(edge_adj, cmap="inferno", ax=ax, annot=True)
ax.set_yticks(ax.get_yticks(), hyperarc_titles, fontsize=12, rotation=0)
ax.set_xticklabels(hyperarc_titles, fontsize=12, rotation=45, ha="left")
ax.xaxis.tick_top()
plt.show()

In [ ]:
hyperarc_centrality = centrality.eigenvector_centrality(
    edge_incidence_matrix,
    hyperarc_weights,
    node_weights,
    rep="dual",
    tolerance=1e-6,
    max_iterations=1000,
    weight_resultant=True,
    random_seed=None,
)

n_conds = n_diseases * [1] + [
    len(d.split(",")) + 1 for d in hyperarc_titles[n_diseases:]
]
hyperarc_evc = pd.DataFrame(
    {
        "Disease": hyperarc_titles,
        "Degree": n_conds,
        "Eigenvector Centrality": hyperarc_centrality,
    }
)
hyperarc_evc.sort_values(by="Eigenvector Centrality", ascending=False).reset_index(
    drop=True
)

In [ ]:
degrees = []
top_progressions = pd.DataFrame(columns=["Disease", "Degree", "Eigenvector Centrality"])
for i, row in hyperarc_evc.sort_values(
    by="Eigenvector Centrality", ascending=False
).iterrows():
    dis, deg, cent = row
    if deg not in degrees:
        degrees.append(deg)
        top_progressions = pd.concat([top_progressions, pd.DataFrame(row).T], axis=0)
top_progressions.reset_index(drop=True)

### Multimorbidity pathway plot using progressive edge adjacency

In [ ]:
# Extract top top_n hyperarcs for up to max_d-degree hyperarcs
top_n = 3
top_progressions = pd.DataFrame()
max_d = 3

for deg in range(1, max_d + 1):
    deg_hyperarc_evc = hyperarc_evc[hyperarc_evc.Degree == deg].sort_values(
        by="Eigenvector Centrality", ascending=False, axis=0
    )
    top_progressions = pd.concat(
        [top_progressions, deg_hyperarc_evc.iloc[:top_n]], axis=0
    )
top_progressions.reset_index(drop=True).sort_values(by="Degree", ascending=True, axis=0)

In [ ]:
# Sort indexing out for plot and rename self-transitiongs to original disease title
rank_progressions = top_progressions.copy(deep=True)
rank_progressions.index = (max_d * [i for i in range(top_n)])[
    : rank_progressions.shape[0]
]
rank_progressions.reset_index(inplace=True)
rank_progressions["Disease"] = [
    prog.split(" -> ")[0] if (prog.split(" -> ")[0] == prog.split(" -> ")[1]) else prog
    for prog in rank_progressions.Disease
]
rank_progressions

In [ ]:
# populate list for every hyperarc where they feed into hyperarcs of degree above
# for plotting purposes below
prog_idx_lists = []

# Loop over degrees
for deg in range(1, max_d):

    # Extract degree specific top ranking hyperarcs
    deg_dis_progs = rank_progressions[rank_progressions.Degree == deg].reset_index(
        drop=True
    )
    lst_idx = deg - 1
    n_degs = deg_dis_progs.shape[0]

    # Initialise list of lists for each top ranking hyperarc of that degree
    prog_idx_lists.append([[] for i in range(n_degs)])

    # Loop over these top ranking hyperarc of degree deg
    for deg_dis_prog in deg_dis_progs.iterrows():
        i, (idx, dis_prog, dis_deg, eig_cent) = deg_dis_prog

        # If not a self-transition, extract tail members and head member
        if deg != 1:
            prog_tail = dis_prog.split(" -> ")[0].split(", ")
            prog_head = dis_prog.split(" -> ")[1]
            prog_dis = np.sort(prog_tail + [prog_head])

        # Otherwise, just extract whole progression as it's one disease
        else:
            prog_dis = dis_prog

        # Extract all top ranking hyperarcs of the degree above
        degabove_dis_progs = rank_progressions[
            rank_progressions.Degree == dis_deg + 1
        ].Disease

        # Loop over these top ranking hyperarcs of the degree above, check if their tail set matches
        # the prog_dis of above, append ranking index to the corresponding list in prog_idx_lists
        for j, degabove_dis in enumerate(degabove_dis_progs):
            degabove_tail = np.sort(degabove_dis.split(" -> ")[0].split(", "))
            if np.all(prog_dis == degabove_tail):
                prog_idx_lists[lst_idx][i].append(j)

In [ ]:
# Line differentiation and progression line colour differentiation
linestyle_lsts = list(matplotlib.lines.lineStyles.keys())[:4]
prog_colors = list(plt.get_cmap("nipy_spectral")(np.linspace(0.1, 0.9, max_d)))

# Plot progressions
sq_s = 10
fig, ax = plt.subplots(1, 1, figsize=(sq_s, sq_s))
sns.scatterplot(
    y="index",
    x="Degree",
    hue="Disease",
    s=200,
    marker="o",
    data=rank_progressions,
    ax=ax,
    legend=False,
)
ax.set_ylabel("Progression Rank", fontsize=15)
ax.set_xlabel("Hyperarc Degree", fontsize=15)
ax.set_yticks(np.arange(top_n))
ax.set_xticks(np.arange(1, max_d + 1))
ax.set_xticklabels(np.arange(1, max_d + 1), fontsize=15)
ax.set_yticklabels(np.arange(1, top_n + 1), fontsize=15)
ax.axis([0, max_d + 1, top_n, -1])
ax.grid("on")

# Include lines to represent degree-to-degree progressions
for i, deg_lst in enumerate(prog_idx_lists):
    for j, idx_list in enumerate(deg_lst):
        for idx in idx_list:
            plt.plot(
                [i + 1, i + 2],
                [j, idx],
                linestyle=linestyle_lsts[j % len(linestyle_lsts)],
                c=prog_colors[i],
                linewidth=2,
            )

# Annotate progressions
for i, dis in enumerate(rank_progressions.Disease):
    row = rank_progressions.iloc[i]
    deg, idx = (row["Degree"], row["index"])
    n_dis = len(dis)
    plt.annotate(
        dis, (deg - (0.5 / 10) * (n_dis / 2), idx - 0.15), fontsize=int(1.5 * sq_s)
    )

fig.tight_layout()
plt.show()